## Example 10 - Lock

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style
import folium

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr, osr

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot

# Used for mathematical functions
import math             
import json

# Used for making the graph to visualize our problem
import networkx as nx 

# Graph location
location_graph = "Shape-Files/Vaarwegvakken"
name_graph = "Vaarwegvakken.shp"

# Start simpy environment
# env = simpy.Environment()
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

### Create graph

The cel below visualizes the problem. In graph theory the red dots are called *edges* and the lines are called *vertices*. Vessels (or any other object) move from edge 1 to edge 3 and from edge 4 to edge 2. The added complexity is that vertice 5-6 only allows traffic in one direction at a time. Vessels can travel simultanously in one direction.

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

In [2]:
FG = nx.read_shp(os.path.join(location_graph, name_graph), simplify=True)

# The read_shp creates a directed graph with single edges
# We require a directed graph but two-way traffic

FG = FG.to_undirected()
FG = FG.to_directed()

In [3]:
def transform_projection(location_graph, name_graph):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataset = driver.Open(os.path.join(location_graph, name_graph))

    # from Layer
    inSpatialRef = dataset.GetLayer().GetSpatialRef()

    # Set up the coordinate reference we want to use, WGS84 - World Geodetic System 1984
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)

    # Transform the coordinates
    transform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    return transform

In [4]:
def change_projection(transform, point):
    point = ogr.CreateGeometryFromWkt(str(point))
    
    point.Transform(transform)
    point.ExportToWkt()
    
    return point.GetX(), point.GetY()

In [5]:
transform = transform_projection(location_graph, name_graph)
FG_new = nx.DiGraph()

nodes_dict = {}

for i, node in enumerate(FG.nodes(data = True)):
    coordinates = change_projection(transform, shapely.geometry.Point(list(FG.nodes)[i][0], list(FG.nodes)[i][1]))
    name = "({:f}, {:f})".format(coordinates[0], coordinates[1])
    geometry = shapely.geometry.Point(coordinates[0], coordinates[1])
    
    nodes_dict[list(FG.nodes)[i]] = name
    FG_new.add_node(name, name = name, Position = coordinates, geometry = geometry, Old = node[1])
    
for edge in FG.edges(data = True):
    node_1 = nodes_dict[edge[0]]
    node_2 = nodes_dict[edge[1]]
    
    VRT_NAAM = edge[2]["VRT_NAAM"]
    VWG_NAAM = edge[2]["VWG_NAAM"]
    BEGKM =  edge[2]["BEGKM"]
    ENDKM =  edge[2]["ENDKM"]
    DIST = np.abs(float(BEGKM) - float(ENDKM))
    
    LINE = (json.loads(edge[2]["Json"])["coordinates"])
    LineString = []
    for coordinates in LINE:
        LineString.append(change_projection(transform, shapely.geometry.Point(coordinates[0], coordinates[1])))
    
    FG_new.add_edge(node_1, node_2, LineString = shapely.geometry.LineString(LineString), 
                    VRT_NAAM = VRT_NAAM, VWG_NAAM = VWG_NAAM, BEGKM = BEGKM, ENDKM = ENDKM, DIST = DIST)

if nx.info(FG) == nx.info(FG_new):
    print("Succes!")

Succes!


### Select only relevant area

In [6]:
# North-East
NE = (4.54, 51.75)
# South-East
SE = (4.54, 51.60)
# South-West
SW = (4.20, 51.60)
# North-West
NW = (4.20, 51.75)

polygon = shapely.geometry.Polygon([NE, SE, SW, NW])

In [7]:
nodes = []
edges = []

for edge in FG_new.edges(data = True):
    node_1 = FG_new.nodes[edge[0]]
    node_2 = FG_new.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        edges.append(edge)

In [8]:
FG_new = nx.DiGraph ()

for node in nodes:
    FG_new.add_node(node["name"], name = node["name"], Position = node["Position"], geometry = node["geometry"])

for edge in edges:
    FG_new.add_edge(edge[0], edge[1], Info = edge[2])

### Show on map

In [9]:
# Browser
m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

for edge in FG_new.edges(data = True):
    points_x = list(edge[2]["Info"]["LineString"].coords.xy[0])
    points_y = list(edge[2]["Info"]["LineString"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Info"]["VWG_NAAM"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)
    
    else:
        folium.PolyLine(line, weight = 2, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)

m

### Create locks
We can see on the maps that there are three locks on the graph, but that the information on the locks is limited. The following edges represent locks:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [10]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

In [11]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]

### Make vessels and paths

In [12]:
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.HasContainer, core.HasResource, core.Routeable), {})

data_vessel_one = {"env": env,
                   "name": "Vessel",
                   "route": ["(4.430289, 51.700047)", "(4.392555, 51.681251)"],
                   "geometry": nx.get_node_attributes(FG_new, "geometry")["(4.430289, 51.700047)"],
                   "capacity": 1_000}

data_vessel_two = {"env": env,
                   "name": "Vessel",
                   "route": ["(4.392555, 51.681251)", "(4.430289, 51.700047)"],
                   "geometry": nx.get_node_attributes(FG_new, "geometry")["(4.392555, 51.681251)"],
                   "capacity": 1_000}

### Start simulation

In [13]:
# Start simpy environment
env.FG = FG_new

# Add the movements of the vessel to the simulation
vessels = []
for i in range(10):
    vessel = Vessel(**data_vessel_one)
    vessels.append(vessel)
    env.process(vessel.move())
    
    vessel = Vessel(**data_vessel_two)
    vessels.append(vessel)
    env.process(vessel.move())

# Run the environment
env.run()

In [14]:
# First ship should be able to directly enter the lock
pd.DataFrame.from_dict(vessels[0].log)

,Message,Timestamp,Value,Geometry
0,Passing lock start,2019-07-18 10:52:35,0,POINT (4.430289385790487 51.70004667570937)
1,Passing lock stop,2019-07-18 11:37:35,2700,POINT (4.392555365264324 51.68125080672722)


In [15]:
# Second ship should be able to pass the lock after the first one has passed
pd.DataFrame.from_dict(vessels[1].log)

,Message,Timestamp,Value,Geometry
0,Waiting in line-up area start,2019-07-18 10:52:35,0.0,POINT (4.392555365264324 51.68125080672722)
1,Waiting in line-up area stop,2019-07-18 11:37:35,2700.0,POINT (4.392555365264324 51.68125080672722)
2,Passing lock start,2019-07-18 11:37:35,0.0,POINT (4.392555365264324 51.68125080672722)
3,Passing lock stop,2019-07-18 12:22:35,2700.0,POINT (4.430289385790487 51.70004667570937)


In [16]:
# Check what the lock is doing
pd.DataFrame.from_dict(lock_nr_1.log).head(10)

,Message,Timestamp,Value,Geometry
0,Lock doors closing start,2019-07-18 10:52:35,"(4.430289, 51.700047)",0
1,Lock doors closing stop,2019-07-18 11:02:35,"(4.430289, 51.700047)",0
2,Lock chamber converting start,2019-07-18 11:02:35,"(4.430289, 51.700047)",0
3,Lock chamber converting stop,2019-07-18 11:27:35,"(4.392555, 51.681251)",0
4,Lock doors opening start,2019-07-18 11:27:35,"(4.392555, 51.681251)",0
5,Lock doors opening stop,2019-07-18 11:37:35,"(4.392555, 51.681251)",0
6,Lock doors closing start,2019-07-18 11:37:35,"(4.392555, 51.681251)",0
7,Lock doors closing stop,2019-07-18 11:47:35,"(4.392555, 51.681251)",0
8,Lock chamber converting start,2019-07-18 11:47:35,"(4.392555, 51.681251)",0
9,Lock chamber converting stop,2019-07-18 12:12:35,"(4.430289, 51.700047)",0


In [17]:
# Fourth ship should be the first one the start waiting at the line-up area
pd.DataFrame.from_dict(vessels[3].log)

,Message,Timestamp,Value,Geometry
0,Waiting in line-up area start,2019-07-18 10:52:35,0.0,POINT (4.392555365264324 51.68125080672722)
1,Waiting in line-up area stop,2019-07-18 11:37:35,2700.0,POINT (4.392555365264324 51.68125080672722)
2,Passing lock start,2019-07-18 11:37:35,0.0,POINT (4.392555365264324 51.68125080672722)
3,Passing lock stop,2019-07-18 12:22:35,2700.0,POINT (4.430289385790487 51.70004667570937)


In [18]:
# Seventh ship should be the first one the start waiting at the waiting area
pd.DataFrame.from_dict(vessels[6].log)

,Message,Timestamp,Value,Geometry
0,Waiting in line-up area start,2019-07-18 10:52:35,0.0,POINT (4.430289385790487 51.70004667570937)
1,Waiting in line-up area stop,2019-07-18 16:52:35,21600.0,POINT (4.430289385790487 51.70004667570937)
2,Passing lock start,2019-07-18 16:52:35,0.0,POINT (4.430289385790487 51.70004667570937)
3,Passing lock stop,2019-07-18 17:37:35,2700.0,POINT (4.392555365264324 51.68125080672722)


In [19]:
# Tenth ship should be the first one the start waiting to enter the waiting area
pd.DataFrame.from_dict(vessels[9].log)

,Message,Timestamp,Value,Geometry
0,Waiting in waiting area start,2019-07-18 10:52:35,0.0,POINT (4.392555365264324 51.68125080672722)
1,Waiting in waiting area stop,2019-07-18 11:37:35,2700.0,POINT (4.392555365264324 51.68125080672722)
2,Waiting in line-up area start,2019-07-18 11:37:35,0.0,POINT (4.392555365264324 51.68125080672722)
3,Waiting in line-up area stop,2019-07-18 13:07:35,5400.0,POINT (4.392555365264324 51.68125080672722)
4,Passing lock start,2019-07-18 13:07:35,0.0,POINT (4.392555365264324 51.68125080672722)
5,Passing lock stop,2019-07-18 13:52:35,2700.0,POINT (4.430289385790487 51.70004667570937)


### Visualize sailed path
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON

In [20]:
import random
def resource_user(name, env, resource, prio):
    req = resource.request(priority=prio)

    yield req
    print("{}, {}".format(name, req.priority))
    yield env.timeout(3)
    resource.release(req)
    
    for i in resource.queue:
        if env.now - 4 > i.time:
            i.priority = -random.random()
    
    print(resource.queue)
    print("")

env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, prio=0))
p2 = env.process(resource_user(2, env, res, prio=0))
p3 = env.process(resource_user(3, env, res, prio=0))
p4 = env.process(resource_user(4, env, res, prio=0))
p5 = env.process(resource_user(5, env, res, prio=-1))
env.run()

1, 0
[<PriorityRequest() object at 0x15e637b6400>, <PriorityRequest() object at 0x15e637b6390>, <PriorityRequest() object at 0x15e637b62b0>, <PriorityRequest() object at 0x15e637b60f0>]

5, -1
[<PriorityRequest() object at 0x15e637b6390>, <PriorityRequest() object at 0x15e637b62b0>, <PriorityRequest() object at 0x15e637b60f0>]

2, -0.09239493970557389
[<PriorityRequest() object at 0x15e637b62b0>, <PriorityRequest() object at 0x15e637b60f0>]

3, -0.008334066746431446
[<PriorityRequest() object at 0x15e637b60f0>]

4, -0.27250151670593403
[]

